In [54]:
from tifffile import imread
import matplotlib.pyplot as plt
from matplotlib.colors import TwoSlopeNorm
import numpy as np
import pandas as pd
from pyama_core.processing.segmentation import segment_cell
from pyama_core.processing.background import correct_bg
from pyama_core.processing.tracking import track_cell
from pyama_core.processing.extraction import extract_trace
from pyama_core.analysis.fitting import fit_model
from pyama_core.analysis.models import get_model

In [6]:
def progress_callback(current, total, message):
    if current % 30 == 0:
        print(f"{message}: {current}/{total}")

In [47]:
phc_good_path = "../data/p000c000.tif"
fluor_good_path = "../data/p000c001.tif"

phc_good = imread(phc_good_path).astype(np.float32, copy=False)
fluor_good = imread(fluor_good_path).astype(np.float32, copy=False)

In [48]:
fig, axs = plt.subplots(1, 2, figsize=(8, 4), constrained_layout=True)
axs[0].imshow(phc_good[100], cmap="gray")
axs[0].set_title("Phase contrast")
axs[1].imshow(fluor_good[100], cmap="hot")
axs[1].set_title("Fluorescence")
axs[0].axis("off")
axs[1].axis("off")
fig.savefig("good.png", dpi=300)

In [49]:
try:
    seg_good = np.load("seg_good.npy")
except FileNotFoundError:
    seg_good = np.empty_like(phc_good, dtype=bool)
    segment_cell(phc_good, seg_good, progress_callback)
    np.save("seg_good.npy", seg_good)

In [50]:
fig, axs = plt.subplots(1, 2, figsize=(8, 4), constrained_layout=True)
axs[0].imshow(phc_good[100], cmap="gray")
axs[0].set_title("Phase contrast")
axs[1].imshow(seg_good[100], cmap="gray")
axs[1].set_title("Segmentation")
axs[0].axis("off")
axs[1].axis("off")
fig.savefig("seg_good.png", dpi=300)

In [51]:
try:
    corrected_good = np.load("corrected_good.npy")
except FileNotFoundError:
    corrected_good = np.empty_like(fluor_good, dtype=np.float32)
    correct_bg(fluor_good, seg_good, corrected_good, progress_callback)
    np.save("corrected_good.npy", corrected_good)

In [58]:
fig, axs = plt.subplots(1, 2, figsize=(8, 4), constrained_layout=True)
vmin = fluor_good.min()
vmax = fluor_good.max()
im0 = axs[0].imshow(
    fluor_good[100],
    cmap="hot",
    norm=TwoSlopeNorm(vmin=vmin, vcenter=vmin + 1000, vmax=vmax),
)
axs[0].set_title("Fluorescence")
im1 = axs[1].imshow(
    corrected_good[100],
    cmap="hot",
    norm=TwoSlopeNorm(vmin=vmin, vcenter=vmin + 1000, vmax=vmax),
)
axs[1].set_title("Background corrected fluorescence")
axs[0].axis("off")
axs[1].axis("off")
fig.colorbar(im0, ax=axs[0], fraction=0.046, pad=0.04)
fig.colorbar(im1, ax=axs[1], fraction=0.046, pad=0.04)
fig.savefig("corrected_good.png", dpi=300)

In [ ]:
try:
    seg_good_labeled = np.load("seg_good_labeled.npy")
except FileNotFoundError:
    seg_good_labeled = np.zeros_like(seg_good, dtype=np.uint16)
    track_cell(seg_good, seg_good_labeled, progress_callback=progress_callback)
    np.save("seg_good_labeled.npy", seg_good_labeled)

Labeling: 0/181
Labeling: 30/181
Labeling: 60/181
Labeling: 90/181
Labeling: 120/181
Labeling: 150/181
Labeling: 180/181
Tracking: 30/181
Tracking: 60/181
Tracking: 90/181
Tracking: 120/181
Tracking: 150/181
Tracking: 180/181


In [19]:
fig, axs = plt.subplots(1, 4, figsize=(8, 2), constrained_layout=True)
time_steps = np.linspace(0, len(seg_good_labeled) - 1, 4).astype(int)
for i, t in enumerate(time_steps):
    fr = seg_good_labeled[t]
    axs[i].imshow(np.where(np.isin(fr, [50, 60, 70]), fr, 0), cmap="hot")
    axs[i].set_title(f"Frame {t}")
    axs[i].axis("off")
fig.savefig("seg_good_labeled.png", dpi=300)

In [24]:
try:
    df = pd.read_csv("trace.csv", index_col=["cell", "time"])
except FileNotFoundError:
    times = np.arange(len(corrected_good)) / 6.0
    df = extract_trace(corrected_good, seg_good_labeled, times, progress_callback)
    df.to_csv("trace.csv")

Extracting features: 0/181
Extracting features: 30/181
Extracting features: 60/181
Extracting features: 90/181
Extracting features: 120/181
Extracting features: 150/181
Extracting features: 180/181


In [30]:
all_cells = df.index.get_level_values("cell").unique()
fig, axs = plt.subplots(1, 2, figsize=(8, 4), constrained_layout=True)
for c in all_cells[20:40]:
    df.loc[c].plot(
        y="intensity_total", ax=axs[0], legend=False, color="green", alpha=0.5
    )
    df.loc[c].plot(y="area", ax=axs[1], legend=False, color="blue", alpha=0.5)
axs[0].set_title("Intensity total")
axs[0].set_xlabel("Time [h]")
axs[0].set_ylim(0, 1e7)
axs[1].set_title("Area")
axs[1].set_ylim(0, 1e4)
axs[1].set_xlabel("Time [h]")
fig.savefig("features.png", dpi=300)

In [38]:
y = df.loc[60]["intensity_total"]
t = df.loc[60].index.values

model = get_model("maturation")
result = fit_model("maturation", t, y)
r_squared = result.r_squared
params = result.fitted_params
y_pred = model.eval(t, params)

In [40]:
fig, axs = plt.subplots(1, 1, figsize=(4, 4), constrained_layout=True)
axs.plot(t, y, label="data")
axs.plot(t, y_pred, label="fit")
axs.text(
    0.05,
    0.95,
    f"$R^2$ = {r_squared:.3f}\n"
    + "\n".join([f"{k} = {v:.3g}" for k, v in params.items()]),
    transform=axs.transAxes,
    fontsize=10,
    verticalalignment="top",
    bbox=dict(boxstyle="round", facecolor="white", alpha=0.8),
)
axs.legend(loc="lower right")
axs.set_xlabel("Time [h]")
axs.set_ylabel("Intensity total")
axs.set_title("Three-stage maturation model")
fig.savefig("fit.png", dpi=300)